In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "15008027" # 店舗ID
SPREADSHEET_KEY = '1-YBOQurssUWZobDZW2Ignoo9zc37RdIz-KmpYp_Q4tQ' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 1201 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "112":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "112":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "561":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "561":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025741', '1', '18462.0', '603.0', '21', '9']
['08/19', '025359', '2', '10256.0', '603.0', '18', '5']
['08/19', '025828', '3', '16667.0', '-4814.0', '20', '7']
['08/19', '025698', '5', '8974.0', '603.0', '9', '2']
['08/19', '025771', '6', '4615.0', '-3610.0', '2', '2']
['08/19', '025892', '7', '11282.0', '-1504.0', '12', '3']
['08/19', '025885', '8', '18462.0', '3010.0', '71', '6']
['08/19', '025862', '10', '10769.0', '603.0', '17', '9']
['08/19', '025647', '11', '22821.0', '8427.0', '55', '16']
['08/19', '025831', '12', '14872.0', '603.0', '17', '6']
['08/19', '025722', '13', '8205.0', '-1203.0', '9', '3']
['08/19', '025712', '15', '10769.0', '302.0', '33', '4']
['08/19', '025874', '16', '9744.0', '3612.0', '16', '3']
['08/19', '025523', '17', '15385.0', '603.0', '22', '6']
['08/19', '025892', '18', '13846.0', '-1203.0', '16', '6']
['08/19', '025828', '20', '36923.0', '-3008.0', '52', '12']
['08/19', '025866', '21', '9231.0', '904.0', '18', '4']
['08/19', '025689', '22', '12

['08/19', '025553', '216', '18205.0', '-8124.0', '6', '3']
['08/19', '025553', '217', '23846.0', '11135.0', '29', '5']
['08/19', '025553', '218', '24359.0', '-4212.0', '13', '7']
['08/19', '025553', '220', '19231.0', '4214.0', '18', '5']
['08/19', '025553', '221', '24359.0', '-2707.0', '13', '4']
['08/19', '025817', '222', '7179.0', '-1504.0', '12', '2']
['08/19', '025817', '223', '13590.0', '4816.0', '52', '9']
['08/19', '025817', '225', '20256.0', '-4212.0', '37', '7']
['08/19', '025817', '226', '10256.0', '-3911.0', '12', '4']
['08/19', '025817', '227', '12821.0', '4816.0', '41', '8']
['08/19', '025863', '228', '11026.0', '-1504.0', '15', '5']
['08/19', '025879', '230', '37692.0', '5418.0', '40', '12']
['08/19', '025544', '231', '8718.0', '904.0', '28', '4']
['08/19', '025876', '232', '14359.0', '3612.0', '13', '3']
['08/19', '025274', '233', '7436.0', '1806.0', '12', '4']
['08/19', '025779', '235', '18974.0', '-14744.0', '2', '2']
['08/19', '025807', '236', '3590.0', '-1203.0', '2'

['08/19', '025775', '528', '27436.0', '4214.0', '19', '6']
['08/19', '025818', '530', '40513.0', '7223.0', '26', '6']
['08/19', '025663', '531', '49744.0', '1806.0', '43', '23']
['08/19', '025864', '532', '25641.0', '-9328.0', '14', '4']
['08/19', '025879', '533', '28974.0', '14746.0', '42', '11']
['08/19', '025850', '535', '16923.0', '-4513.0', '7', '2']
['08/19', '025680', '536', '37179.0', '-2406.0', '20', '7']
['08/19', '025867', '537', '53590.0', '-28587.0', '10', '6']
['08/19', '025707', '538', '21282.0', '3612.0', '20', '7']
['08/19', '025878', '550', '18974.0', '-12036.0', '8', '1']
['08/19', '025826', '551', '34103.0', '-12036.0', '11', '5']
['08/19', '025890', '552', '47949.0', '42431.0', '92', '7']
['08/19', '025825', '553', '40769.0', '-20462.0', '10', '6']
['08/19', '025825', '555', '26923.0', '12038.0', '23', '5']
['08/19', '025825', '556', '37949.0', '-23170.0', '7', '3']
['08/19', '025606', '557', '40769.0', '-19559.0', '13', '8']
['08/19', '025606', '558', '0', '0', '0

['08/19', '025665', '750', '42051.0', '-19860.0', '11', '3']
['08/19', '025665', '751', '42821.0', '-17453.0', '15', '6']
['08/19', '025665', '752', '36154.0', '4214.0', '25', '7']
['08/19', '025665', '753', '43077.0', '904.0', '26', '6']
['08/19', '025665', '755', '34103.0', '-9027.0', '15', '5']
['08/19', '025665', '756', '32821.0', '3311.0', '21', '5']
['08/19', '025665', '757', '16410.0', '-12638.0', '2', '2']
['08/19', '025665', '758', '28205.0', '14445.0', '25', '3']
['08/19', '025665', '760', '26667.0', '-17754.0', '6', '5']
['08/19', '025665', '761', '30000.0', '-1504.0', '17', '3']
['08/19', '025665', '762', '0', '0', '0', '0']
['08/19', '025665', '763', '29744.0', '-15346.0', '7', '1']
['08/19', '025665', '765', '30000.0', '-4814.0', '15', '4']
['08/19', '025680', '766', '10000.0', '-7522.0', '2', '2']
['08/19', '025814', '767', '0', '0', '0', '0']
['08/19', '025764', '768', '13590.0', '-9629.0', '1', '1']
['08/19', '025825', '770', '12051.0', '4816.0', '10', '1']
['08/19', '

['08/19', '025867', '1066', '7949.0', '14144.0', '15', '2']
['08/19', '025867', '1067', '11282.0', '5117.0', '10', '3']
['08/19', '025867', '1068', '23077.0', '-9930.0', '6', '3']
['08/19', '025867', '1070', '13333.0', '11436.0', '17', '5']
['08/19', '120110', '1201', '3839', '2130.0', '32', '15', '--']
['08/19', '120110', '1202', '4789', '-972.0', '23', '21', '--']
['08/19', '120110', '1203', '5613', '-2269.0', '28', '12', '--']
['08/19', '120110', '1205', '2916', '-1343.0', '13', '5', '--']
['08/19', '120110', '1206', '4787', '-463.0', '28', '13', '--']
['08/19', '120110', '1207', '4465', '5000.0', '47', '22', '--']
['08/19', '120110', '1208', '3801', '-3241.0', '11', '5', '--']
['08/19', '120110', '1210', '6198', '-2593.0', '29', '16', '--']
['08/19', '120110', '1211', '4205', '-556.0', '25', '8', '--']
['08/19', '120110', '1212', '5173', '-1204.0', '27', '12', '--']
['08/19', '120110', '1213', '5075', '-3148.0', '20', '9', '--']
['08/19', '120110', '1215', '3102', '4907.0', '41', '

['08/19', '120125', '1388', '3474', '-1111.0', '9', '6', '--']
['08/19', '120125', '1500', '6341', '185.0', '24', '15', '--']
['08/19', '120125', '1501', '5823', '2037.0', '29', '17', '--']
['08/19', '120125', '1502', '4990', '-185.0', '19', '10', '--']
['08/19', '120125', '1503', '6017', '-648.0', '19', '14', '--']
['08/19', '120125', '1505', '6006', '-1389.0', '18', '12', '--']
['08/19', '120125', '1506', '8665', '880.0', '34', '17', '--']
['08/19', '120125', '1507', '5942', '1852.0', '30', '10', '--']
['08/19', '120125', '1508', '7461', '-694.0', '29', '12', '--']
['08/19', '120125', '1510', '2896', '-1389.0', '5', '7', '--']
['08/19', '120125', '1511', '5900', '-1204.0', '16', '13', '--']
['08/19', '120125', '1512', '8049', '3843.0', '41', '25', '--']
['08/19', '120125', '1513', '5872', '-1481.0', '14', '19', '--']
['08/19', '120125', '1515', '3972', '-926.0', '12', '8', '--']
['08/19', '120125', '1516', '6939', '1620.0', '32', '19', '--']
['08/19', '120125', '1517', '5224', '-556.

['08/19', '120077', '1701', '1408', '-139.0', '0', '4', '11']
['08/19', '120077', '1702', '0', '0', '0', '0', '--']
['08/19', '120077', '1703', '4213', '-1250.0', '0', '8', '32']
['08/19', '120126', '1705', '5689', '-417.0', '0', '14', '31']
['08/19', '120126', '1706', '1821', '-1944.0', '0', '4', '3']
['08/19', '120126', '1707', '5051', '833.0', '0', '10', '32']
['08/19', '120126', '1708', '2919', '-1713.0', '0', '8', '8']
['08/19', '120121', '1710', '648', '-185.0', '0', '2', '2']
['08/19', '120121', '1711', '1798', '-1343.0', '0', '3', '14']
['08/19', '120121', '1712', '279', '-185.0', '0', '1', '3']
['08/19', '120121', '1713', '237', '-278.0', '0', '0', '--']
['08/19', '120093', '1715', '64', '0', '0', '0', '--']
['08/19', '120093', '1716', '2246', '-1389.0', '7', '0', '7']
['08/19', '120093', '1717', '532', '-556.0', '1', '0', '1']
['08/19', '120093', '1718', '987', '-880.0', '1', '0', '1']
['08/19', '120093', '1720', '147', '-231.0', '0', '0', '--']
['08/19', '120093', '1721', '1

{'spreadsheetId': '1-YBOQurssUWZobDZW2Ignoo9zc37RdIz-KmpYp_Q4tQ',
 'updates': {'spreadsheetId': '1-YBOQurssUWZobDZW2Ignoo9zc37RdIz-KmpYp_Q4tQ',
  'updatedRange': 'S5RAW!F1380'}}